In [1]:
import numpy as np
import sys
import os
import argparse

import pickle
import fnmatch
from tqdm import tqdm
from utils import save_dir, analyze_dir, results_dir,train_pool,save_dict

from tqdm import tqdm
import re
import itertools

In [2]:
model_identifier = 'NN-tree_nclass=64_nobj=64000_nhier=6_beta=0.0923671_sigma=5.0_nfeat=936-train_test-fixed'
model_identifier = 'NN-tree_nclass=64_nobj=64000_nhier=6_beta=0.000161_sigma=5.0_nfeat=936-train_test-fixed'
train_identifier = 'epochs-10_batch-32_lr-0.01_momentum-0.5_init-gaussian_std-1e-06'
analyze_identifier='mftma-exm_per_class=50-proj=False-rand=True-kappa=1e-08-n_t=300-n_rep=5'
#epochs-10_batch-32_lr-0.001_momentum-0.5_init-gaussian_std-0.0001
#                           epochs-10_batch-32_lr-0.002_momentum-0.6_init-gaussian_std-1e-05 \
#                           epochs-10_batch-32_lr-0.01_momentum-0.5_init-gaussian_std-1e-06 

In [3]:
training_files = []
for file in os.listdir(os.path.join(save_dir, model_identifier, train_identifier)):
    if fnmatch.fnmatch(file, '*.pth'):
        training_files.append(os.path.join(save_dir, model_identifier, train_identifier, file))

In [4]:
grad_pkl_files = []
for file in os.listdir(os.path.join(save_dir, analyze_identifier, model_identifier, train_identifier)):
    if fnmatch.fnmatch(file, '*gradient_data_v3.pkl'):
        grad_pkl_files.append(os.path.join(save_dir, analyze_identifier, model_identifier, train_identifier, file))

Load extracted gradient data (from extraction script!)

In [5]:
s = [re.findall('/\d+', x) for x in grad_pkl_files]
s = [item for sublist in s for item in sublist]
dummy_id = [(x.split('/')) for x in s]
file_id = [int(x.split('/')[1]) for x in s]
sorted_files = [grad_pkl_files[x] for x in np.argsort(file_id)]
grad_pkl_files = sorted_files

In [6]:
params = train_pool[model_identifier]()
params.load_dataset()
layer_names = params.get_layer_names()[1:]
print(layer_names)
transfo_mat = params.dataset.transformation_mats
tiled_transfo_mat = [np.tile(x, (1, 50)).reshape(-1, x.shape[1]) for x in transfo_mat]


ERROR:root:ERROR: not a MATLAB datatype: <HDF5 dataset "data": shape (128252,), type "|u1">, (uint8)
ERROR:root:ERROR: not a MATLAB datatype: <HDF5 dataset "ir": shape (128252,), type "<u8">, (uint64)
ERROR:root:ERROR: not a MATLAB datatype: <HDF5 dataset "jc": shape (64128,), type "<u8">, (uint64)
ERROR:root:ERROR: MATLAB type not supported: graph, (uint32)
ERROR:root:ERROR: MATLAB type not supported: graph, (uint32)


['layer_1_Linear', 'layer_2_Linear', 'layer_3_Linear']


In [7]:
layer_names = params.get_layer_names()[1:]
layer_names

['layer_1_Linear', 'layer_2_Linear', 'layer_3_Linear']

In [8]:
epoch_data=[]
for idx, file in tqdm(enumerate(grad_pkl_files)):
        g = pickle.load(open(file, 'rb'))
        batch_=dict()
        for key in g['results'][0].keys():
            e = np.asarray(g['results'][0][key]).squeeze()
            batch_[key]=(np.reshape(e, [-1, e.shape[2]]))
        epoch_data.append(batch_)


1060it [08:20,  2.12it/s]


In [10]:
all_e_f_list=dict()
for layer in layer_names:
    e_f_=[x[layer] for x in epoch_data]
    all_e_f_list[layer]=e_f_

(3200, 64)

In [11]:
layer_gradient_dict=dict()
for layer in layer_names:
    print(f"analyzing {layer} \n")
    layer_branch_data=dict()
    all_grad_data = []
    e_f_list = all_e_f_list[layer]

    for hier_id,transfo in enumerate(tiled_transfo_mat):
        print(f"analyzing hierarchy {hier_id} \n")
        # get grad data:
        # create combination pairs
        example_per_class=int(np.unique(transfo.sum(axis=0)))
        a = list(range(example_per_class))
        combs = list(itertools.combinations_with_replacement(a, r=2))
        combs_1 = list(itertools.combinations(a, r=2))
        combs_1 = [(x[1], x[0]) for x in combs_1] #flip
        all_combs = [combs, combs_1]
        all_combs = [item for sublist in all_combs for item in sublist]
        #if len(all_combs)>example_per_class**2:
        select_combs = [all_combs[x] for x in np.random.choice(np.arange(len(all_combs)), size=1500)]
        #else:
        #    select_combs=all_combs
        # use the combination to compute the differences in vectors
        all_branch_diffs = []
        for _, time_point in tqdm(enumerate(e_f_list)):
            branch_diffs = []
            for branch in transfo.transpose():  # for each single "over" branch, iterate through the categories
                P12 = time_point[np.where(branch == 1), :].squeeze()  # extract each category
                p12_diff = [np.diff(P12[x, :], axis=0) for x in
                            select_combs]  # find all pairwise differences between all samples in cat 1 and cat 2 (the leaf branches)
                branch_diffs.append(
                    np.mean(np.stack(p12_diff).squeeze(), axis=0))  # take mean over all the pairwise diffs
            all_branch_diffs.append(branch_diffs)
        # compute the norms for differences
        branch_norm = [[np.linalg.norm(x) for x in branch_diffs] for branch_diffs in all_branch_diffs]
        branch_norm_mat = np.stack(branch_norm)
        layer_branch_data[f"hier_{hier_id}"]=branch_norm_mat
    layer_gradient_dict[layer]=layer_branch_data
    # save layer data independently
    layer_gradient_file = os.path.join(save_dir, analyze_identifier, model_identifier, train_identifier,
                                 f'{model_identifier}_{layer}_gradient_pooled_v3_large.pkl')
    d_layer = {'analyze_identifier': analyze_identifier,
                'model_identifier': model_identifier,
                'train_identifier': train_identifier,
                'layer':layer,
                'gradient_results': layer_branch_data}
    save_dict(d_layer, layer_gradient_file)
    print('saved ' + layer_gradient_file)


0it [00:00, ?it/s]

analyzing layer_1_Linear 

analyzing hierarchy 0 



1060it [21:29,  1.22s/it]
0it [00:00, ?it/s]

analyzing hierarchy 1 



1060it [09:07,  1.94it/s]
0it [00:00, ?it/s]

...

analyzing hierarchy 4 



1060it [00:58, 18.07it/s]


analyzing hierarchy 5 



1060it [00:29, 35.88it/s]

saved /mindhive/evlab/u/Shared/Greta_Eghbal_manifolds/extracted/mftma-exm_per_class=50-proj=False-rand=True-kappa=1e-08-n_t=300-n_rep=5/NN-tree_nclass=64_nobj=64000_nhier=6_beta=0.000161_sigma=5.0_nfeat=936-train_test-fixed/epochs-10_batch-32_lr-0.01_momentum-0.5_init-gaussian_std-1e-06/NN-tree_nclass=64_nobj=64000_nhier=6_beta=0.000161_sigma=5.0_nfeat=936-train_test-fixed_layer_3_Linear_gradient_pooled_v3_large.pkl


Make a transformation matrix that does not average across num examples